In [ ]:
import sys
import os
from datetime import datetime
import numpy as np
import pandas as pd
import json
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
import pickle as cp
from tensorflow import keras
import tensorflow as tf
from keras import backend as K
import datetime as dt
from tensorflow.keras.layers.experimental.preprocessing import Normalization



In [ ]:
def addSpanLag(data, events=[], look_back=7, look_ahead = 7, season=0):##span==1
#     ### Adding span (required for weekly, monthly average data)
#     if span>1:
#         nts = []
#         ind = 0
#         while ind < len(ts):
#             tmpts = ts[ind:ind+span]
#             print (tmpts)
#             dt = tmpts[0]
#             vals = [k for k in tmpts]
#             pr = sum(vals)/len(vals)
#             nts.append((dt,pr))
#             ind+=span
#         ts = nts
#     if not convert_to_Xy:
#         return ts

    ### Adding lag (based on look back and look ahead)
    
    if len(events) > 0:
        input = np.append(data,events,axis=1)
    else:
        input = data
        
    if season>0:
        season = np.array([k%season for k in range(data.shape[0])])
        season = season.reshape(len(season),1)
        input = np.append(input,season,axis=1)

    ind = 0
    X_data = []
    y_data = []

    while ind + look_back + look_ahead < len(data):
        
        X = input[ind:ind+look_back]
        Y = data[ind+look_back+look_ahead]
        X_data.append(X)
        y_data.append(Y)
        ind+=1
    X_data = np.array(X_data)
    y_data = np.array(y_data)

    return X_data, y_data

In [ ]:
def getConfig(model_name):
    params = [k for k in model_name.split('_')]
    print(params)
    prm_dict = {}
    prm_dict['crop'] = params[1].split('=')[1]
    prm_dict['normalization'] = params[2].split('=')[1]
    prm_dict['season'] = int(params[-1].split('=')[1])
    prm_dict['type'] = params[0]
    return prm_dict
    

In [ ]:
diff=False
fileformat = 'food_price_data/%s.csv'
train_split = 2557 # -1 # if using diff

In [ ]:
model_path = 'oct14'
models = os.listdir(model_path)
results = []

for model_name in models:
    print(model_name)

    params = getConfig(model_name)
    crop = params['crop']
    model_type = params['type']
    normalization = params['normalization']
    season = params['season']
    trn = model_name.split('_')[-1]
    
    if normalization == 'False':
        continue
    
    food_price = pd.read_csv(fileformat%(crop))['price']
    
    if diff:
        food_price = food_price.diff().dropna() 
    price_diff = np.array(food_price).reshape(food_price.shape[0],1)

    mean = np.mean(price_diff)
    std = np.std(price_diff)
    if normalization:
        price_diff = (price_diff - mean)/std

    ### Events
    events_train = np.load('embeddings/events_train.npy')
    events_test = np.load('embeddings/events_test.npy')
    events = np.append(events_train,events_test, axis=0)

    ### Topics
    topics_train = np.load('embeddings/toi.maxLDA_train.npy')
    topic_test = np.load('embeddings/toi.maxLDA_test.npy')
    topics = np.append(topics_train,topic_test, axis=0)

    ### Word2Vec
    w2v_train = np.load('embeddings/toi.W2V_train.npy')
    w2v_test = np.load('embeddings/toi.W2V_test.npy')
    w2v = np.append(w2v_train,w2v_test, axis=0)

    
    X_event, y_event   = addSpanLag(price_diff,events,look_back=7,look_ahead=6,season=season)
    X_event_train = X_event[:train_split]
    y_event_train = y_event[:train_split]
    X_event_test = X_event[train_split:]
    y_event_test = y_event[train_split:]

    X_topics, y_topics   = addSpanLag(price_diff,topics,look_back=7,look_ahead=6,season=season)
    X_topics_train = X_topics[:train_split]
    y_topics_train = y_topics[:train_split]
    X_topics_test = X_topics[train_split:]
    y_topics_test = y_topics[train_split:]

    X_w2v, y_w2v   = addSpanLag(price_diff,w2v,look_back=7,look_ahead=6,season=season)
    X_w2v_train = X_w2v[:train_split]
    y_w2v_train = y_w2v[:train_split]
    X_w2v_test = X_w2v[train_split:]
    y_w2v_test = y_w2v[train_split:]


    X_noevent, y_noevent   = addSpanLag(price_diff,look_back=7,look_ahead=6,season=season)
    X_noevent_train = X_noevent[:train_split]
    y_noevent_train = y_noevent[:train_split]
    X_noevent_test = X_noevent[train_split:]
    y_noevent_test = y_noevent[train_split:]



    model = keras.models.load_model(os.path.join(model_path,model_name))
   

    if model_type == 'ren':
        e_tr_acc = model.evaluate(X_event_train,y_event_train)
        e_tst_acc_13_20 = model.evaluate(X_event_test,y_event_test)
    elif model_type == 'lda':
        e_tr_acc = model.evaluate(X_topics_train,y_topics_train)
        e_tst_acc_13_20 = model.evaluate(X_topics_test,y_topics_test)
    elif model_type == 'w2v':
        e_tr_acc = model.evaluate(X_w2v_train,y_w2v_train)
        e_tst_acc_13_20 = model.evaluate(X_w2v_test,y_w2v_test)
    else:
        e_tr_acc = model.evaluate(X_noevent_train,y_noevent_train)
        e_tst_acc_13_20 = model.evaluate(X_noevent_test,y_noevent_test)
    
    
    if normalization:
        e_tr_acc = (e_tr_acc *std)
        e_tst_acc_13_20 = (e_tst_acc_13_20 *std)
           
    op = "%s,%s,%s,%.3f,%.3f"\
    %(crop,model_type,season, e_tr_acc,e_tst_acc_13_20)
    
    results.append(op)


In [ ]:
print('crop,model_type,season,training,testing')
for r in sorted(results):
    print(r)